# Notes and Cheatsheet

This is a summary of all the lessons learned so far.

In order to run applications we write in TypeScript, we need to compile the TypeScript files into JavaScript. We can either do this once with the `tsc` command, or we can watch (recursively) the current working directory with `tsc --watch` or `tsc -w`.

In [33]:
// import the typechecker function so that we can demo type checking inline
import { checkTypeScript } from './utils/typechecker.ts';


## A note on type checking

TypeScript does not prevent us from running code that will error at runtime.


In [34]:
const double = (x: number): number => x * 2;
double('anything');


NaN

We're still able to call `double('anything')` and get `NaN` in response, just as we would with JavaScript. It's only when we compile or use a typechecker (as we do with the `checkTypeScript` function) that we see the error for a non-number argument value.


In [35]:
await checkTypeScript(`const double = (x: number): number => x * 2; double('anything');`);

error: TS2345 [ERROR]: Argument of type 'string' is not assignable to parameter of type 'number'.
const double = (x: number): number => x * 2; double('anything');
                                                    ~~~~~~~~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/fb0dc9419505c323.ts:1:53


Our IDE may also provide type hints to help us.

## Variables

A variable has a type. The type is separated from the variable name with `:`.

In [36]:
// type annotations
let sum: number = 1 + 2;
sum;

3

In [37]:
// check the type of sum
await checkTypeScript(`let sum: number = 1 + 2;`);


Types are correct.


In [38]:
// type annotations
let sum: number = 'any' + 'thing';
sum;

"anything"

In [39]:
// type errors
await checkTypeScript(`let sum: number = 'any' + 'thing';`);


error: TS2322 [ERROR]: Type 'string' is not assignable to type 'number'.
let sum: number = 'any' + 'thing';
    ~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/48af2d794d55453c.ts:1:5


## Functions

A function takes arguments, which have types. The function returns something, which also has a type. The types are marked with `:`, as they were for simple variables.

In [40]:
function add(a: number, b: number): number {
  return a + b;
}

In [41]:
add(1, 2);


3

In [42]:
await checkTypeScript(`function add(a: number, b: number): number { return a + b; }; add(1, 2);`);


Types are correct.


**NOTE**: if we don't set a type explicitly for our params then they will default to `any`.

**NOTE**: we need to declare the type for each param, even if two or more share the same type. For example, the following will make the type of `a` equal to `any` _not_ `number`:

In [43]:
await checkTypeScript(`function add(a, b: number): number { return a + b; }; add(1, 2);`);


error: TS7006 [ERROR]: Parameter 'a' implicitly has an 'any' type.
function add(a, b: number): number { return a + b; }; add(1, 2);
             ^
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/f17a333676d255e5.ts:1:14


### This is what TypeScript was designed for

If we pass `true` in to the add function it appears to work, we get the number `2` as a result, but this is because JavaScript implicitly converts `true` to `1` (and `false` to `0`).

In [44]:
add(1, true)  // true is implicitly converted to 1 by the JavaScript runtime

2

However, with the type checker (or compiler), will return an error because we're trying to use a boolean instead of a number.

In [45]:
await checkTypeScript(`function add(a: number, b: number): number { return a + b; }; add(1, true);`);


error: TS2345 [ERROR]: Argument of type 'boolean' is not assignable to parameter of type 'number'.
function add(a: number, b: number): number { return a + b; }; add(1, true);
                                                                     ~~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/a33b77d7743d5cc7.ts:1:70


## Operators


In [46]:
// these should run just fine
console.log(1 + 1);
console.log(2 * 2);
console.log('1' + '1'); // TypeScript allows string concatenation with `+`

// NOTE: TypeScript will also allow mixed concatenation of strings and numbers
console.log('1' + 1);
console.log(1 + '1');

// these should throw type errors but will "work" at runtime
console.log('2' * '2');
console.log('4' / '2');
console.log(1 / '1');
console.log('1' / 1);


2
4
11
11
11
4
2
1
1


In [47]:
// these should check just fine
await checkTypeScript(`console.log(1 + 1);`);
await checkTypeScript(`console.log(2 * 2);`);
await checkTypeScript(`console.log('1' + '1');`); // TypeScript allows string concatenation with `+`

// NOTE: TypeScript will also allow mixed concatenation of strings and numbers
await checkTypeScript(`console.log('1' + 1);`);
await checkTypeScript(`console.log(1 + '1');`);

// these should throw type errors
await checkTypeScript(`console.log('2' * '2');`);
await checkTypeScript(`console.log('4' / '2');`);
await checkTypeScript(`console.log(1 / '1');`);
await checkTypeScript(`console.log('1' / 1);`);


Types are correct.
Types are correct.
Types are correct.
Types are correct.
Types are correct.
error: TS2362 [ERROR]: The left-hand side of an arithmetic operation must be of type 'any', 'number', 'bigint' or an enum type.
console.log('2' * '2');
            ~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/9709d4e42c4ed469.ts:1:13

TS2363 [ERROR]: The right-hand side of an arithmetic operation must be of type 'any', 'number', 'bigint' or an enum type.
console.log('2' * '2');
                  ~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/9709d4e42c4ed469.ts:1:19

Found 2 errors.
error: TS2362 [ERROR]: The left-hand side of an arithmetic operation must be of type 'any', 'number', 'bigint' or an enum type.
console.log('4' / '2');
            ~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/2d10366797a0d8e8.ts:1:13

TS2363 [ERROR]: The right-hand side of an arithmetic operation must be of type 'any', 'number', 'bigint' or an enum ty

# Types

Types can be defined using the `type` keyword. Types are idiomatically named using `UpperCamelCase`.

In [48]:
type MyStringType = string;
let myString: MyStringType = 'Hello, world!';
myString;

"Hello, world!"

## The "types" of types

TypeScript has different types of types that it can use. [This page](https://www.typescriptlang.org/docs/handbook/2/everyday-types.html) in the TypeScript docs explains them.


## Type Erasure

The TypeScript compiler works by reading through a TypeScript file and looking for any type annotations. It checks any types it finds and then **removes the TypeScript code** turning it into plain JavaScript.

This means that our runtime code contains no TypeScript, so we can't do things like inspect types at runtime.

## Generic Types

We can declare our own generic types as follows:

In [49]:
// Let's model a pair of pants with left and right pockets...

// ... in this instance, both pockets must contain the same type, but that type can be anything we want
await checkTypeScript(`
  type Pants<T> = {left: T, right: T};
  let myPants: Pants<number> = {left: 1, right: 2}; myPants;
`);

await checkTypeScript(`
  type Pants<T> = {left: T, right: T};
  let myPants: Pants<string> = {left: 'phone', right: 'wallet'}; myPants;
`);

await checkTypeScript(`
  type Pants<T> = {left: T, right: T};
  let myPants: Pants<number> = {left: 1, right: 'wallet'}; myPants;
`);

Types are correct.
Types are correct.
error: TS2322 [ERROR]: Type 'string' is not assignable to type 'number'.
  let myPants: Pants<number> = {left: 1, right: 'wallet'}; myPants;
                                         ~~~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/f37ddf9891e2e0ea.ts:3:42

    The expected type comes from property 'right' which is declared here on type 'Pants<number>'
      type Pants<T> = {left: T, right: T};
                                ~~~~~
        at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/f37ddf9891e2e0ea.ts:2:29


In [50]:
// Now let's model a pair of pants with left and right pockets that can contain different types...
await checkTypeScript(`
  type Pants<T1, T2> = {left: T1, right: T2};
  let myPants: Pants<number, number> = {left: 1, right: 2}; myPants;
`);

await checkTypeScript(`
  type Pants<T1, T2> = {left: T1, right: T2};
  let myPants: Pants<number, string> = {left: 1, right: 'wallet'}; myPants;
`);

await checkTypeScript(`
  type Pants<T1, T2> = {left: T1, right: T2};
  let myPants: Pants<number, string> = {left: 1, right: 2}; myPants;
`);

Types are correct.
Types are correct.
error: TS2322 [ERROR]: Type 'number' is not assignable to type 'string'.
  let myPants: Pants<number, string> = {left: 1, right: 2}; myPants;
                                                 ~~~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/1b1d67f01caa242a.ts:3:50

    The expected type comes from property 'right' which is declared here on type 'Pants<number, string>'
      type Pants<T1, T2> = {left: T1, right: T2};
                                      ~~~~~
        at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/1b1d67f01caa242a.ts:2:35


In [51]:
// Using rather than typechecking...
type Pants<T1, T2> = {left: T1, right: T2};
let myPants: Pants<number, string> = {left: 1, right: 'wallet'};
myPants;

{ left: 1, right: "wallet" }

## Union Types

Sometimes we want to allow multiple type options to be available. For this, we can use **union types**. A union type says "the type can either be a or b or c".

In [52]:
// Given a union type of string | number, we can assign a string or a number to a variable of that type
await checkTypeScript(`let myVar: string | number = 'a string';`);
await checkTypeScript(`let myVar: string | number = 42;`);

// However, we can't assign any other type, such as a boolean
await checkTypeScript(`let myVar: string | number = true;`);

Types are correct.
Types are correct.
error: TS2322 [ERROR]: Type 'boolean' is not assignable to type 'string | number'.
let myVar: string | number = true;
    ~~~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/aeb8b9a1b3ca497f.ts:1:5


## Optional Types

Sometimes we want to have optional arguments to a function. How might we annotate those?

In [53]:
// If we annotate the type based on what we expect to be passed in,
// we will fail a type check if we don't pass in both arguments
await checkTypeScript(`
  function concat(first: string, last: string): string {
    if (!last) {
      return first;
    }

    return first + ' ' + last;
  }

  // this will run just fine
  concat('hello', 'world');

  // this will throw a type error
  concat('hello');
`);

error: TS2554 [ERROR]: Expected 2 arguments, but got 1.
  concat('hello');
  ~~~~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/ec886a05dc7e070b.ts:14:3

    An argument for 'last' was not provided.
      function concat(first: string, last: string): string {
                                     ~~~~~~~~~~~~
        at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/ec886a05dc7e070b.ts:2:34


In [54]:
// We can use an option type to allow for undefined values
// (note we use `?:` rather than `:` in the function signature)
await checkTypeScript(`
  function concat(first: string, last?: string): string {
    if (!last) {
      return first;
    }

    return first + ' ' + last;
  }

  // these both work fine now
  concat('hello', 'world');
  concat('hello');
`);

Types are correct.


The optional type is not just syntactic sugar for `string | undefined` as I first thought. Using that type signature would mean we would have to call the function like `concat("hello", undefined)`. Calling `concat("hello")` would still fail the type check.

In [55]:
await checkTypeScript(`
  function concat(first: string, last: string | undefined): string {
    if (!last) {
      return first;
    }

    return first + ' ' + last;
  }

  // this will throw a type error because the function still expects two arguments
  concat('hello');
`);

error: TS2554 [ERROR]: Expected 2 arguments, but got 1.
  concat('hello');
  ~~~~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/8ebfe935177241c3.ts:11:3

    An argument for 'last' was not provided.
      function concat(first: string, last: string | undefined): string {
                                     ~~~~~~~~~~~~~~~~~~~~~~~~
        at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/8ebfe935177241c3.ts:2:34


We _can_ still pass in `undefined` if we want to when using optional types though...

In [56]:
await checkTypeScript(`
  function concat(first: string, last?: string): string {
    if (!last) {
      return first;
    }

    return first + ' ' + last;
  }

  // these will both work fine
  concat('hello');
  concat('hello', undefined);
`);

Types are correct.


## Type Guards

If a function uses **union types** we can optionally further set type guarantees within the function by using **type guards**.

In the following example `Array.isArray()` is a type guard when used in an `if`. It guarantees that the code within its block will only run if `userOrUsers` is an array. Similarly, the `else` will only run if `userOrUsers` is _not_ an array. TypeScript knows that these amount to type guards. The `if` block only runs for type `User[]` and the else block only runs for type `User`.

In [57]:
await checkTypeScript(`
  type User = { name: string };

  function nameOrLength(userOrUsers: User | User[]): string | number {
    if (Array.isArray(userOrUsers)) {
      // Inside this side of the if, userOrUsers' type is User[].
      return userOrUsers.length;
    } else {
      // Inside this side of the if, userOrUsers' type is User.
      return userOrUsers.name;
    }
  }

  const userOrUsers: User | User[] = [{name: 'Amir'}];
  // we cannot call name on userOrUsers because it could be an array
  userOrUsers.name;
`);

await checkTypeScript(`
  type User = { name: string };

  function nameOrLength(userOrUsers: User | User[]): string | number {
    if (Array.isArray(userOrUsers)) {
      // Inside this side of the if, userOrUsers' type is User[].
      return userOrUsers.length;
    } else {
      // Inside this side of the if, userOrUsers' type is User.
      return userOrUsers.name;
    }
  }

  const userOrUsers: User | User[] = {name: 'Amir'};
  // we cannot call length on userOrUsers because it could be a User
  userOrUsers.length;
`);

await checkTypeScript(`
  type User = { name: string };

  function nameOrLength(userOrUsers: User | User[]): string | number {
    if (Array.isArray(userOrUsers)) {
      // Inside this side of the if, userOrUsers' type is User[].
      return userOrUsers.length;
    } else {
      // Inside this side of the if, userOrUsers' type is User.
      return userOrUsers.name;
    }
  }

  // however we can call nameOrLength with a user or an array of users and it will work
  nameOrLength({name: 'Amir'});
  nameOrLength([{name: 'Amir'}, {name: 'Betty'}]);
`);

type User = { name: string };

function nameOrLength(userOrUsers: User | User[]): string | number {
  if (Array.isArray(userOrUsers)) {
    // Inside this side of the if, userOrUsers' type is User[].
    return userOrUsers.length;
  } else {
    // Inside this side of the if, userOrUsers' type is User.
    return userOrUsers.name;
  }
}

const name: string = nameOrLength({name: 'Amir'});
const length: number = nameOrLength([{name: 'Amir'}, {name: 'Betty'}]);

`name: ${name}, length: ${length}`;

error: TS2339 [ERROR]: Property 'name' does not exist on type 'User[]'.
  userOrUsers.name;
              ~~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/499f554cf49ed543.ts:16:15
error: TS2339 [ERROR]: Property 'length' does not exist on type 'User'.
  userOrUsers.length;
              ~~~~~~
    at file:///var/folders/kw/gm1hxsgn0xd5x25trdphvhpm0000gn/T/9f873a16af52e71b.ts:16:15
Types are correct.


"name: Amir, length: 2"